<a href="https://colab.research.google.com/github/1digitaldesign/sample-code/blob/main/ETL_%26_SQL_Tai%2CBetty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Setup

Part 1. ETL

Part 2. SQL 

## Part 1: ETL


In [14]:
#@title Installation of Necessary Packages 
%%capture
!pip3 install pyspark
!pip3 install hide_code
!pip3 install fancyimpute
!pip3 install miceforest
!pip3 install scikit-learn
%pip install mlxtend --upgrade

On the left, download the dataset [`usa_00001.csv`](https://drive.google.com/file/d/128-eo8QVTYu20aRzA_-7OTxPOjyS-SRp/view?usp=share_link) and on the left hand side, there is a folder to click on called `Files` and then `Upload to Session Storage` this file [`usa_00001.csv`](https://drive.google.com/file/d/128-eo8QVTYu20aRzA_-7OTxPOjyS-SRp/view?usp=share_link). 

In [15]:
#@title Installation of Necessary Libraries 
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import sqlite3
import requests
import os
from sklearn.impute import SimpleImputer
from mlxtend.evaluate import bias_variance_decomp

## Data Extraction

In [4]:
df = pd.read_csv('/content/usa_00001.csv')

## Data Transformation
Data transformation is an integral step for data analysis and modeling without errors and improve the performance of the data. 

* Cleaning 
* Normalization
* Reduction
* Discretization
* Aggregation






In [5]:
df.isnull().sum()
df.shape

(3252599, 32)

In [6]:
for col_name in df.columns: 
    print(col_name)

YEAR
SAMPLE
SERIAL
CBSERIAL
HHWT
CLUSTER
STRATA
GQ
PERNUM
PERWT
SEX
AGE
MARST
BIRTHYR
MARRNO
DIVINYR
WIDINYR
SCHOOL
EDUC
EDUCD
DEGFIELD
DEGFIELDD
DEGFIELD2
DEGFIELD2D
EMPSTAT
EMPSTATD
OCC2010
IND
INDNAICS
WKSWORK1
FTOTINC
INCWAGE


In [7]:
df.describe().style.highlight_max(color="darkred")

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STRATA,GQ,PERNUM,PERWT,SEX,AGE,MARST,BIRTHYR,MARRNO,DIVINYR,WIDINYR,SCHOOL,EDUC,EDUCD,DEGFIELD,DEGFIELDD,DEGFIELD2,DEGFIELD2D,EMPSTAT,EMPSTATD,OCC2010,IND,WKSWORK1,FTOTINC,INCWAGE
count,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000,3252599.000000
mean,2021.000000,202101.000000,717559.681547,2021001168653.933838,99.464870,2021007175597.814209,466571.456261,1.126290,2.065684,102.039552,1.508804,42.617305,3.610626,1978.382695,0.763501,0.589478,0.590393,1.192275,6.333877,65.788119,11.555902,1157.102767,1.288970,129.084694,1.554270,15.577713,6478.958507,3790.784774,22.357544,605649.415975,196215.597491
std,0.000000,0.000000,419092.279450,2064895.535001,86.706876,4190922.794503,942077.458427,0.559180,1.332517,89.786774,0.499923,23.877383,2.308894,23.877383,0.777666,0.502382,0.504037,0.457692,3.268188,32.530554,21.115907,2113.806014,7.932513,794.273182,1.131287,11.300173,3518.795781,3735.829994,24.694715,2166826.373888,366775.670771
min,2021.000000,202101.000000,1.000000,2021000000001.000000,1.000000,2021000000011.000000,10001.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1925.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,-17000.000000,0.000000
25%,2021.000000,202101.000000,349456.500000,2021000372398.000000,48.000000,2021003494566.000000,82408.000000,1.000000,1.000000,49.000000,1.000000,22.000000,1.000000,1959.000000,0.000000,0.000000,0.000000,1.000000,5.000000,50.000000,0.000000,0.000000,0.000000,0.000000,1.000000,10.000000,3630.000000,0.000000,0.000000,40000.000000,0.000000
50%,2021.000000,202101.000000,715274.000000,2021000744727.000000,75.000000,2021007152741.000000,210334.000000,1.000000,2.000000,76.000000,2.000000,43.000000,4.000000,1978.000000,1.000000,1.000000,1.000000,1.000000,6.000000,65.000000,0.000000,0.000000,0.000000,0.000000,1.000000,10.000000,7010.000000,3470.000000,1.000000,81000.000000,23000.000000
75%,2021.000000,202101.000000,1085054.500000,2021001116914.000000,120.000000,2021010850546.000000,410308.000000,1.000000,3.000000,124.000000,2.000000,62.000000,6.000000,1999.000000,1.000000,1.000000,1.000000,1.000000,10.000000,101.000000,19.000000,1903.000000,0.000000,0.000000,3.000000,30.000000,9920.000000,7860.000000,52.000000,149500.000000,90000.000000
max,2021.000000,202101.000000,1440865.000000,2021010160574.000000,2798.000000,2021014408651.000000,7030155.000000,5.000000,20.000000,3223.000000,2.000000,96.000000,6.000000,2021.000000,3.000000,2.000000,2.000000,2.000000,11.000000,116.000000,64.000000,6403.000000,64.000000,6403.000000,3.000000,30.000000,9920.000000,9920.000000,52.000000,9999999.000000,999999.000000


In [8]:
df["FTOTINC"].value_counts()

9999999    163874
0           47529
50000       27362
40000       25954
60000       25804
            ...  
232590          1
330510          1
128060          1
1428000         1
507840          1
Name: FTOTINC, Length: 31326, dtype: int64

In [9]:
df["INCWAGE"].value_counts()

0         1159371
999999     549819
50000       48682
40000       46840
30000       46459
           ...   
448000          1
468000          1
435000          1
406000          1
388000          1
Name: INCWAGE, Length: 1009, dtype: int64

In [5]:
df_nan = df.copy()
df_nan["FTOTINC"] = df["FTOTINC"].replace(9999999, np.nan)
df_nan["INCWAGE"] = df["INCWAGE"].replace(999999, np.nan)

df_nan.max()

YEAR                   2021
SAMPLE               202101
SERIAL              1440865
CBSERIAL      2021010160574
HHWT                   2798
CLUSTER       2021014408651
STRATA              7030155
GQ                        5
PERNUM                   20
PERWT                  3223
SEX                       2
AGE                      96
MARST                     6
BIRTHYR                2021
MARRNO                    3
DIVINYR                   2
WIDINYR                   2
SCHOOL                    2
EDUC                     11
EDUCD                   116
DEGFIELD                 64
DEGFIELDD              6403
DEGFIELD2                64
DEGFIELD2D             6403
EMPSTAT                   3
EMPSTATD                 30
OCC2010                9920
IND                    9920
INDNAICS             999920
WKSWORK1                 52
FTOTINC           2508000.0
INCWAGE            787000.0
dtype: object

### Imputation of missing data 
There are several techniques we may impute the missing data, and we will examine the bias-variance tradeoff using the [MLxtend](http://rasbt.github.io/mlxtend/) Library. 


*   Substitution by mean
*   Substitution by median
*   Model Imputation
*   Maximum Likelihood Imputation
*   Multiple Imputation 

Papers: 

*   Kang H. [The prevention and handling of the missing data.](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3668100/) Korean J Anesthesiol. 2013 May;64(5):402-6. doi: 10.4097/kjae.2013.64.5.402. Epub 2013 May 24. PMID: 23741561; PMCID: PMC3668100.
*   Allison, Paul D. “[Handling Missing Data by Maximum Likelihood.](https://statisticalhorizons.com/wp-content/uploads/MissingDataByML.pdf)” (2012). 




#### Substitution by mean 
I opted not to split the data into train and test, but I could have. I am
treating the NaN data as the target variable and doing simple imputation. 

In [12]:
df_nan.isnull().sum()

YEAR               0
SAMPLE             0
SERIAL             0
CBSERIAL           0
HHWT               0
CLUSTER            0
STRATA             0
GQ                 0
PERNUM             0
PERWT              0
SEX                0
AGE                0
MARST              0
BIRTHYR            0
MARRNO             0
DIVINYR            0
WIDINYR            0
SCHOOL             0
EDUC               0
EDUCD              0
DEGFIELD           0
DEGFIELDD          0
DEGFIELD2          0
DEGFIELD2D         0
EMPSTAT            0
EMPSTATD           0
OCC2010            0
IND                0
INDNAICS           0
WKSWORK1           0
FTOTINC       163874
INCWAGE       549819
dtype: int64

In [22]:
from sklearn.impute import SimpleImputer
train_mean = df_nan[['BIRTHYR','FTOTINC','INCWAGE']].copy()
mean_imputer = SimpleImputer(strategy='mean')
train_mean.iloc[:,:] = mean_imputer.fit_transform(train_mean)
train_mean.isnull().sum()
df_mean = df_nan.fillna(train_mean)
df_mean.isnull().sum()

<ipython-input-22-cbb6edf2d5a8>:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  train_mean.iloc[:,:] = mean_imputer.fit_transform(train_mean)


YEAR          0
SAMPLE        0
SERIAL        0
CBSERIAL      0
HHWT          0
CLUSTER       0
STRATA        0
GQ            0
PERNUM        0
PERWT         0
SEX           0
AGE           0
MARST         0
BIRTHYR       0
MARRNO        0
DIVINYR       0
WIDINYR       0
SCHOOL        0
EDUC          0
EDUCD         0
DEGFIELD      0
DEGFIELDD     0
DEGFIELD2     0
DEGFIELD2D    0
EMPSTAT       0
EMPSTATD      0
OCC2010       0
IND           0
INDNAICS      0
WKSWORK1      0
FTOTINC       0
INCWAGE       0
dtype: int64

#### Substitution by median

In [23]:
from sklearn.impute import SimpleImputer
train_median = df_nan[['BIRTHYR','FTOTINC','INCWAGE']].copy()
median_imputer = SimpleImputer(strategy='median')
train_median.iloc[:,:] = median_imputer.fit_transform(train_median)
train_median.isnull().sum()
df_median = df_nan.fillna(train_median)
df_median.isnull().sum()

<ipython-input-23-fe080b6983d3>:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  train_median.iloc[:,:] = median_imputer.fit_transform(train_median)


YEAR          0
SAMPLE        0
SERIAL        0
CBSERIAL      0
HHWT          0
CLUSTER       0
STRATA        0
GQ            0
PERNUM        0
PERWT         0
SEX           0
AGE           0
MARST         0
BIRTHYR       0
MARRNO        0
DIVINYR       0
WIDINYR       0
SCHOOL        0
EDUC          0
EDUCD         0
DEGFIELD      0
DEGFIELDD     0
DEGFIELD2     0
DEGFIELD2D    0
EMPSTAT       0
EMPSTATD      0
OCC2010       0
IND           0
INDNAICS      0
WKSWORK1      0
FTOTINC       0
INCWAGE       0
dtype: int64

#### Model Imputation

#### Maximum Likelihood Imputation

#### Multiple Imputation

## Part 2: Creating the database -- SQL 

In [24]:
conn = sqlite3.connect('censusData.db')
df_median.to_sql('censusDatabase', con = conn, if_exists='replace', index=False)
pd.read_sql('SELECT * FROM censusDatabase WHERE AGE > "40"', con = conn).head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STRATA,GQ,PERNUM,PERWT,...,DEGFIELD2,DEGFIELD2D,EMPSTAT,EMPSTATD,OCC2010,IND,INDNAICS,WKSWORK1,FTOTINC,INCWAGE
0,2021,202101,1,2021010000026,13,2021000000011,80001,3,1,13,...,0,0,3,30,9920,0,0,0,76800.0,0.0
1,2021,202101,2,2021010000031,51,2021000000021,80001,3,1,51,...,0,0,3,30,9920,0,0,0,76800.0,0.0
2,2021,202101,3,2021010000063,17,2021000000031,120001,3,1,17,...,0,0,3,30,9920,0,0,0,76800.0,0.0
3,2021,202101,5,2021010000100,15,2021000000051,50001,3,1,15,...,0,0,3,30,9920,0,0,0,76800.0,0.0
4,2021,202101,9,2021010000401,71,2021000000091,120001,3,1,71,...,0,0,3,30,9920,0,0,0,76800.0,0.0


References: 

**Data Set**

Steven Ruggles, Sarah Flood, Matthew Sobek, Danika Brockman, Grace Cooper,  Stephanie Richards, and Megan Schouweiler. IPUMS USA: Version 13.0 [dataset]. Minneapolis, MN: IPUMS, 2023. https://doi.org/10.18128/D010.V13.0